In [10]:
import os
from dotenv import load_dotenv
load_dotenv('../Langchain/.env')


os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_KEY']=os.getenv("LANGCHAIN_KEY")
os.environ['LANGCHAIN_PROJECT']=os.getenv("LANGCHAIN_PROJECT")
os.environ['LANGCHAIN_TRACKING_V2']='true'

In [11]:
from langchain_community.document_loaders import WebBaseLoader

In [13]:
loader=WebBaseLoader(web_path="https://en.wikipedia.org/wiki/Periyar")
docs=loader.load()
docs

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Periyar', 'title': 'Periyar - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nPeriyar - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nBio

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
documents=splitter.split_documents(docs)


In [18]:
from langchain_community.embeddings import OllamaEmbeddings
embd=OllamaEmbeddings(model='gemma2:2b')


In [19]:
from langchain_community.vectorstores import FAISS

vectordb=FAISS.from_documents(documents,embedding=embd)


In [24]:
res=vectordb.similarity_search('why periyar ? ')
res[0].page_content

'Gandhi accepted karma in the sense that "the Untouchables reap the reward of their karma,[118] but was against discrimination against them using the revaluing term Harijans. As shown in the negotiations at Vaikom his methods for abolishing discrimination were: to stress on the orthodox, inhumane treatment of Untouchables; to secure voluntary lifting of the ban by changing the hearts of caste Hindus; and to work within a Hindu framework of ideas.[118]\nOn the Temple Entry issue, pro-Periyar sources claim that Gandhi never advocated the opening of Garbha Griha to Harijans in consequence of his Hindu belief. These pro-Periyar sources, with the exception of M. Mahar and D.S. Sharma, clearly show that Periyar and his followers emphasised that Periyar was the real fighter for the removal of Untouchability and the true upliftment of Hairjans, whereas Gandhi was not. This did not prevent Periyar from having faith in Gandhi on certain matters.[118]'

In [31]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-3o")

In [32]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
    <context>
        {context}
    </context>

"""
)

doc_chain=create_stuff_documents_chain(llm,prompt)
doc_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n    <context>\n        {context}\n    </context>\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002750D2D5720>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002750D2D7D60>, root_client=<openai.OpenAI object at 0x000002750B042C80>, root_async_client=<openai.AsyncOpenAI object at 0x000002750D2D56F0>, model_name='gpt-3o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kw

In [ ]:
from langchain_core.documents import Document
query='Erode Venkata Ramasamy was born '
doc_chain.invoke({
    "input":query,
    "context":[Document(page_content=query)]
})

In [36]:
#Retriever 
retriever=vectordb.as_retriever()
from langchain.chains import create_retrieval_chain

retriever_chain=create_retrieval_chain(retriever,doc_chain)


In [ ]:
retriever_chain.invoke(
    {
        "input":query,
    }
)